In [ ]:
#%pip install mediapipe

In [ ]:
import mediapipe as mp
from matplotlib import pyplot as plt
import random
import glob
from util import draw_landmarks_on_image
from image_landmarker import ImageLandmarker
from livestream_landmarker import LivestreamLandmarker
import cv2 as cv

In [ ]:
# extract dataset from zip
import zipfile
with zipfile.ZipFile("american-sign-language-letters.v1i.coco.zip","r") as zip_ref:
    zip_ref.extractall("american-sign-language-letters.v1i.coco")

In [ ]:
def image_detector(image):
    try:
        detector = ImageLandmarker()
        detection_result = detector.detect(image)
    finally: 
        detector.close()
                    # method in util.py
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    plt.axis('off')
    plt.imshow(annotated_image)

In [ ]:
# prepare from image
def get_random_image():
    filename = random.choice(glob.glob('./american-sign-language-letters.v1i.coco/train/*.jpg'))
    return mp.Image.create_from_file(filename)

# # Load the input image from a numpy array.
# mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_image)

In [ ]:
image_detector(get_random_image())

In [ ]:
def livestream_detector():
    try:
        cap = cv.VideoCapture(0)
        detector = LivestreamLandmarker()
        if not cap.isOpened():
            print("Cannot open camera")
            exit()
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            # if frame is read correctly ret is True
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            detector.detect_async(mp_image)
            annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detector.result)
            cv.imshow('frame', annotated_image)
            # 1000/50 = 20 FPS
            if cv.waitKey(100) == ord('q'):
                break
    finally:    
        # When everything done, release the capture
        detector.close()
        cap.release()
        cv.destroyAllWindows()

In [ ]:
livestream_detector()